# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 12:24PM,264976,18,SEQ-9586196,Seqirus,Released
1,Jun 19 2023 12:08PM,264975,20,9586271,Else Nutrition,Released
2,Jun 19 2023 11:59AM,264973,10,Enova-11980,Emerginnova,Released
3,Jun 19 2023 11:59AM,264973,10,Enova-11981,Emerginnova,Released
4,Jun 19 2023 11:49AM,264970,10,MSP222787,Methapharm-G,Released
5,Jun 19 2023 11:47AM,264968,19,9586241,"Innogenix, LLC",Released
6,Jun 19 2023 11:47AM,264968,19,9586242,"Innogenix, LLC",Released
7,Jun 19 2023 11:47AM,264968,19,9586247,"Innogenix, LLC",Released
8,Jun 19 2023 11:47AM,264968,19,9586248,"Innogenix, LLC",Released
9,Jun 19 2023 11:47AM,264968,19,9586249,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,264968,Released,25
24,264970,Released,1
25,264973,Released,2
26,264975,Released,1
27,264976,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264968,NaN,NaN,25.0
264970,NaN,NaN,1.0
264973,NaN,NaN,2.0
264975,NaN,NaN,1.0
264976,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,3.0,0.0,1.0
264925,0.0,1.0,0.0
264926,1.0,24.0,1.0
264927,0.0,1.0,0.0
264928,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264921,3,0,1
264925,0,1,0
264926,1,24,1
264927,0,1,0
264928,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,3,0,1
1,264925,0,1,0
2,264926,1,24,1
3,264927,0,1,0
4,264928,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264921,3,,1
1,264925,,1,
2,264926,1,24,1
3,264927,,1,
4,264928,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 12:24PM,264976,18,Seqirus
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition
2,Jun 19 2023 11:59AM,264973,10,Emerginnova
4,Jun 19 2023 11:49AM,264970,10,Methapharm-G
5,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC"
30,Jun 19 2023 11:39AM,264967,10,Methapharm-G
31,Jun 19 2023 11:32AM,264964,10,"Citieffe, Inc."
33,Jun 19 2023 10:53AM,264960,20,"Exact-Rx, Inc."
38,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc."
46,Jun 19 2023 10:50AM,264958,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 19 2023 12:24PM,264976,18,Seqirus,,,1
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition,,,1
2,Jun 19 2023 11:59AM,264973,10,Emerginnova,,,2
3,Jun 19 2023 11:49AM,264970,10,Methapharm-G,,,1
4,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC",,,25
5,Jun 19 2023 11:39AM,264967,10,Methapharm-G,,,1
6,Jun 19 2023 11:32AM,264964,10,"Citieffe, Inc.",,1,1
7,Jun 19 2023 10:53AM,264960,20,"Exact-Rx, Inc.",,,5
8,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc.",,3,5
9,Jun 19 2023 10:50AM,264958,20,"Exact-Rx, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 12:24PM,264976,18,Seqirus,1,,
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1,,
2,Jun 19 2023 11:59AM,264973,10,Emerginnova,2,,
3,Jun 19 2023 11:49AM,264970,10,Methapharm-G,1,,
4,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC",25,,
5,Jun 19 2023 11:39AM,264967,10,Methapharm-G,1,,
6,Jun 19 2023 11:32AM,264964,10,"Citieffe, Inc.",1,1,
7,Jun 19 2023 10:53AM,264960,20,"Exact-Rx, Inc.",5,,
8,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc.",5,3,
9,Jun 19 2023 10:50AM,264958,20,"Exact-Rx, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 12:24PM,264976,18,Seqirus,1,,
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1,,
2,Jun 19 2023 11:59AM,264973,10,Emerginnova,2,,
3,Jun 19 2023 11:49AM,264970,10,Methapharm-G,1,,
4,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC",25,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 12:24PM,264976,18,Seqirus,1.0,NaN,NaN
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1.0,NaN,NaN
2,Jun 19 2023 11:59AM,264973,10,Emerginnova,2.0,NaN,NaN
3,Jun 19 2023 11:49AM,264970,10,Methapharm-G,1.0,NaN,NaN
4,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC",25.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 12:24PM,264976,18,Seqirus,1.0,0.0,0.0
1,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1.0,0.0,0.0
2,Jun 19 2023 11:59AM,264973,10,Emerginnova,2.0,0.0,0.0
3,Jun 19 2023 11:49AM,264970,10,Methapharm-G,1.0,0.0,0.0
4,Jun 19 2023 11:47AM,264968,19,"Innogenix, LLC",25.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1059874,5.0,1.0,0.0
15,1059804,41.0,45.0,1.0
16,264954,0.0,1.0,0.0
18,264976,1.0,0.0,0.0
19,1324674,27.0,26.0,1.0
20,1324748,11.0,12.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1059874,5.0,1.0,0.0
1,15,1059804,41.0,45.0,1.0
2,16,264954,0.0,1.0,0.0
3,18,264976,1.0,0.0,0.0
4,19,1324674,27.0,26.0,1.0
5,20,1324748,11.0,12.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,5.0,1.0,0.0
1,15,41.0,45.0,1.0
2,16,0.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,27.0,26.0,1.0
5,20,11.0,12.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,5.0
1,15,Released,41.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,0.0,1.0,0.0,0.0,1.0,3.0
Executing,1.0,45.0,1.0,0.0,26.0,12.0
Released,5.0,41.0,0.0,1.0,27.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,0.0,1.0,0.0,0.0,1.0,3.0
1,Executing,1.0,45.0,1.0,0.0,26.0,12.0
2,Released,5.0,41.0,0.0,1.0,27.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,0.0,1.0,0.0,0.0,1.0,3.0
1,Executing,1.0,45.0,1.0,0.0,26.0,12.0
2,Released,5.0,41.0,0.0,1.0,27.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()